In [3]:
!pip install git+https://github.com/openai/swarm.git
!pip install openai
!pip install firecrawl-py

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-we93a6jb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-we93a6jb
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 30.0 MB/s eta 0:00:00
  Created wheel for swarm: filename=swarm-0.1.0-py3-none-any.whl size=25999 sha256=965de2633a9d

In [4]:
import os
from firecrawl import FirecrawlApp
from swarm import Agent, Swarm
from openai import OpenAI
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from openai import ChatCompletion

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "schema" in "FirecrawlApp.ExtractParams" shadows an attribute in parent "BaseModel"
  warnings.warn(


In [11]:
# Configure API Keys

os.environ["OPENAI_API_KEY"]= ""
#Load dataset
df = pd.read_csv("https://raw.githubusercontent.com/jaydiaz2012/AI_First_Day_6_AI_Swarm/refs/heads/main/ai%20first%20sales%20data%20-%20sales%20(1).csv")

# Analyze dataset
def analyze_dataset():
    df['date'] = pd.to_datetime(df['date'])
    numerical_df = df.select_dtypes(include=np.number)

    summary = {
        "overview": df.describe(include="all").to_dict(),
        "null_values": df.isnull().sum().to_dict(),
        "correlations": numerical_df.corr().to_dict(),
    }
    return summary

def generate_web_insights():
    web_insights = {
        'source_pageviews': df.groupby('source')['pageviews'].sum(),
        'source_visits': df.groupby('source')['visits'].sum(),
        'source_transactions': df.groupby('source')['transactions'].sum(),
        'source_product_click': df.groupby('source')['productClick'].sum()
    }
    return web_insights

def generate_web_visuals():
    visualizations = {}

    # Page Views Distribution
    plt.figure(figsize=(10, 6))
    sns.lineplot(df, x="source", y="pageviews")
    plt.title("Page Views Distribution")
    visualizations['page_views_distribution']
    plt.show()

    # Visits Distribution
    plt.figure(figsize=(10, 6))
    sns.lineplot(df, x="source", y="visits")
    plt.title("Visits Distribution")
    visualizations['visits_distribution']
    plt.show()

    # Transactions Trends
    plt.figure(figsize=(10, 6))
    sns.lineplot(df, x="source", y="transactions")
    plt.title("Transactions Trends")
    visualizations['transactions_trends']
    plt.show()

    # Product Click Trends
    plt.figure(figsize=(10, 6))
    sns.lineplot(df, x="source", y="productClick")
    plt.title("Product Clicks Trends")
    visualizations['product_click_trends']
    plt.show()

    return visualizations

web_analytics_agent = Agent(
    name="Web Analyst Agent",
    model="gpt-4o-mini",
    instructions="""You are a web analyst agent that cleans the dataset and report on the statistical summary of the dataset""",
    functions=[analyze_dataset],
)

web_insights_agent = Agent(
    name="Web Insights Agent",
    model="gpt-4o-mini",
    instructions="""You are a Web Insights Agent that provide insights about the data provided and summarize the insights from other agents, including actionable recommendations""",
    functions=[generate_web_insights]
)

web_insights_visuals_agent = Agent(
    name="Web_Visuals_Agent",
    model="gpt-4o-mini",
    instructions="""You are a data visualization expert that create visualizations on a given datase and gives summary of the data visuals""",
    fuctions=[generate_web_visuals]
)

if __name__ == "__main__":
    client = Swarm()

    web_analytics_response = client.run(
        agent=web_analytics_agent,
        messages=[{
            "role": "user",
            "content": "Please clean and analyze my dataset."
        }]
    )

    print("\nWeb Analytics Results:")
    print(web_analytics_response.messages[-1]["content"])

    web_insights_response = client.run(
        agent=web_insights_agent,
        messages=[{
            "role": "user",
            "content": "Please provide insights based from my dataset."
        }]
    )

    print("\nWeb Insights Results:")
    print(web_insights_response.messages[-1]["content"])

    web_insights_visuals_response = client.run(
        agent=web_insights_visuals_agent,
        messages=[{
            "role": "user",
            "content": "Please provide charts using Seaborn or Matplotlib using the data from the given dataset."
        }]
    )

    print("\nWeb Analytics Results:")
    print(web_insights_visuals_response.messages[-1]["content"])

# Advanced analysis using Swarm
def advanced_analysis(prompt, model):
    try:
        prompt = (
            "Perform clustering, detect anomalies, and provide predictive trends "
            f"on this dataset:\n{df.to_csv(index=False)}"
        )
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a data analysis assistant using Swarm."},
                {"role": "user", "content": prompt}
            ],
        )
        return response.choices[0].message['content']
    except Exception as e:
        st.error(f"Error performing advanced analysis: {e}")
        return None

# Generate business insights
def generate_insights(summary, model):
    prompt = (
        f"Here is the dataset summary:\n{summary}\n"
        "Generate actionable business insights from the above dataset."
    )
    response = model.create(
        engine="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a data analyst AI."},
            {"role": "user", "content": prompt}
        ],
    )
    return response.choices[0].message['content']

# Generate a report
def generate_report(advanced_analysis, generate_insights, generate_web_visuals, file_path="report.txt"):
    with open(file_path, "w") as file:
        file.write("Automated Business Insights Report\n")
        file.write("=" * 50 + "\n\n")
        file.write("**Insights:**\n")
        file.write(insights + "\n\n")
        file.write("**Advanced Analysis:**\n")
        file.write(advanced_analysis_result + "\n")
    return file_path



Web Analytics Results:
Here's the statistical summary and analysis of your dataset based on the provided data:

### Overview

#### Date
- **Count**: 52,721
- **Mean**: 2020-05-19 14:22:09
- **Min**: 2020-01-01
- **Max**: 2020-09-30

#### Source
- **Count**: 52,721
- **Unique**: 22
- **Top Source**: Facebook (12,954 occurrences)

#### Medium
- **Count**: 52,721
- **Unique**: 6
- **Top Medium**: CPC (16,833 occurrences)

#### Delivery Available
- **Count**: 52,721
- **Unique**: 3
- **Top Value**: No data (31,953 occurrences)

#### Device Type
- **Count**: 52,721
- **Unique**: 3
- **Top Device**: Mobile (24,416 occurrences)

#### Promo Activated
- **Count**: 52,721
- **Unique**: 2
- **Top Value**: No (27,548 occurrences)

#### Filter Used
- **Count**: 52,721
- **Unique**: 2
- **Top Value**: No (37,290 occurrences)

### Page Interaction Metrics
- **Pageviews**: 
  - **Mean**: 583.76
  - **Min**: 0
  - **Max**: 34,832
  - **Std Dev**: 1,452.00

- **Visits**: 
  - **Mean**: 127.12
  - **Min